In [ ]:
import numpy as np
import keras
from keras import backend as K
from keras.layers import Activation
from keras.layers.core import Dense, Flatten

from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train_path="/content/drive/MyDrive/train"
valid_path='/content/drive/MyDrive/valid'
test_path='/content/drive/MyDrive/test'

In [ ]:
from keras.models import Sequential

Imported Dense from keras.layers, to perform the full connection of the neural network.

Importing Flatten to perform flattening step in order to get a single long continuous linear vector.

Importing Maxpooling function to perform pooling operation, since we need the maximum value pixel from the respective region of interest.

The term “dropout” refers to dropping out units (both hidden and visible) in a neural network. Simply put, dropout refers to ignoring units (i.e. neurons) during the training phase of certain set of neurons which is chosen at random.

In [ ]:
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D

Before fitting images to the neural network, we need to synthesize the training data i.e. images. We will use keras.preprocessing library for this task to prepare the images in the training set as well as the test set.
Image augmentation helps us increase the scope of our very limited dataset. Already existing photos can be manipulated via techniques such as shearing, rotation, zooming, flipping etc., to generate new samples and extend our dataset without having to download more images.
Below, we will define two objects of the ImageDataGenerator class to augment our existing set of images.
The training dataset will be further augmented with new samples through rescaling, shearing, zooming and horizontal flipping and we will also resize all images to have height and width dimensions of 300.

In [ ]:
train_batches = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True, vertical_flip=True, rescale=1/255., shear_range=0.1, rotation_range= 30).flow_from_directory(train_path, target_size=(224,224), classes=['Email','Memo','News','Report','Resume'], batch_size=32)
valid_batches = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True, vertical_flip=True, rescale=1/255., shear_range=0.1, rotation_range= 20).flow_from_directory(valid_path, target_size=(224,224), classes=['Email','Memo','News','Report','Resume'], batch_size=32)
test_batches = ImageDataGenerator(rescale=1/255.).flow_from_directory(test_path, target_size=(224,224), classes=['Email','Memo','News','Report','Resume'], batch_size=10)

Found 100 images belonging to 5 classes.
Found 15 images belonging to 5 classes.
Found 10 images belonging to 5 classes.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from keras.applications import VGG16
from keras import layers, models, backend as K

In [ ]:
K.clear_session()
base_model = VGG16(include_top=False, weights='imagenet',input_shape=(224,224,3))

In [ ]:
for l in base_model.layers:
  l.trainable = False

In [ ]:
x = layers.Flatten()(base_model.output)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.6)(x)
x = layers.Dense(200, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(100, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(50, activation='relu')(x)
out = layers.Dense(5, activation='softmax')(x)
model_new = models.Model(base_model.input, out)
model_new.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

Also called logarithmic loss, log loss or logistic loss. Each predicted class probability is compared to the actual class desired output 0 or 1 and a score/loss is calculated that penalizes the probability based on how far it is form the actual expected value. The penalty is logarithmic in nature yielding a large score for large differences close to 1 and small score for small differences tending to 0.
Cross-entropy loss is used when adjusting model weights during training. The aim is to minimize the loss, i.e, the smaller the loss the better the model. A perfect model has a cross-entropy loss of 0.

In [ ]:
model_new.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['acc'])
hist = model_new.fit_generator(train_batches, steps_per_epoch=3, validation_data=valid_batches, validation_steps=1, epochs=100, verbose=2)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
3/3 - 61s - loss: 0.8330 - acc: 0.6979 - val_loss: 1.7258 - val_acc: 0.4000
Epoch 2/100
3/3 - 58s - loss: 0.7736 - acc: 0.6979 - val_loss: 1.8509 - val_acc: 0.4000
Epoch 3/100
3/3 - 45s - loss: 0.6507 - acc: 0.7647 - val_loss: 2.1923 - val_acc: 0.4000
Epoch 4/100
3/3 - 45s - loss: 0.6949 - acc: 0.7353 - val_loss: 2.1569 - val_acc: 0.4000
Epoch 5/100
3/3 - 59s - loss: 0.6062 - acc: 0.7812 - val_loss: 2.1134 - val_acc: 0.4000
Epoch 6/100
3/3 - 45s - loss: 0.8727 - acc: 0.6324 - val_loss: 1.8596 - val_acc: 0.4667
Epoch 7/100
3/3 - 45s - loss: 0.6954 - acc: 0.7500 - val_loss: 1.9712 - val_acc: 0.4000
Epoch 8/100
3/3 - 45s - loss: 0.7247 - acc: 0.7647 - val_loss: 1.9938 - val_acc: 0.4000
Epoch 9/100
3/3 - 45s - loss: 0.5832 - acc: 0.7941 - val_loss: 1.9086 - val_acc: 0.4667
Epoch 10/100
3/3 - 45s - loss: 0.6798 - acc: 0.7647 - val_loss: 2.1020 - val_acc: 0.3333
Epoch 11/100
3/3 - 58s - loss: 0.5920 - acc: 0.7500 - val_loss: 2.0532 - val_acc: 0.4000
Epoch 12/100
3/3 - 45s - loss:

In [ ]:
import numpy as np
test_imgs, test_labels = next(test_batches)
print(test_labels)
y_test= [ np.where(r==1)[0][0] for r in test_labels ]
y_test

[[0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]]


[3, 1, 4, 4, 1, 3, 0, 0, 2, 2]

In [ ]:
p = model_new.predict(test_imgs)

In [ ]:
p.argmax(axis=1)

array([1, 1, 1, 4, 1, 1, 1, 2, 2, 2])

In [ ]:
sum(p.argmax(axis=1) == y_test)/10*100

50.0